In [2]:
!pip install PyPDF2

You are using pip version 9.0.3, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [53]:
import PyPDF2 as pypdf
import regex as re
import numpy as np
import csv
import pandas as pd

In [54]:
PDFfile = open("dinecollege2019-2020.pdf", "rb")
pdfread = pypdf.PdfFileReader(PDFfile)
pdfread

In [56]:
y = pdfread.getPage(93)
y.extractText().replace('\n', '')

'94     SWO 242   Social  Work Practice I   (3)   This course develops generalist social work knowledge and skills for working with and on behalf of  individuals, families, and groups. It will look  at identifying practice skills through the assessment and  treatment phases.   SWO 243  Human Behavior in the  Social  Environment   (3)   This course   develops an understanding of human behavior on a biological and behavioral level. It looks  at  and various methods o f treatment and engages students in issues related to trauma, spiritual  development, disabilities, and bridging the gap of micro and macro levels of practice.   SWO 250 Social Work Ethics   (3)   This course   is designed to educate students on legal and ethical v alues related Social Work. The course  will educate students on the NASW code of ethics and laws related to social work practice. It will also  provide a look at the social worker role in the legal system.     SWO 295 Field Experience I (3)   Prerequisite: ENG 10 

In [6]:
pdfread.getNumPages()

94

In [47]:
def get_data(csv_name, pdf_name, page_range, pattern, dep_group, abbrev_group, c_group, t_group,
            year, grad_status):
    """Takes in name of PDF in local folder and extracts data in given PAGE_RANGE 
    using regex expression PATTERN and given group numbers. YEAR is the catalog year.
    GRAD_STATUS is either None (if unknown), 'G', 'UG', or a number indicating at
    which number is the course graduate."""
    
    PDFfile = open(pdf_name, "rb")
    pdfread = pypdf.PdfFileReader(PDFfile)
    
    csv_writer = csv.writer(open(csv_name, 'w'))
    csv_writer.writerow(["Department Name", "Course Catalogue Number", "Course Name",
     "Course Description", "Graduate/Undergraduate", "Format", "Lab", 
     "Academic Catalogue Year"])
    FS_keys = ["agri", "food", "animal"]
    
    look_ahead = pattern + r'(.+?)(?=' + pattern + r')'
    
    dep_names = {}
    for i in np.arange(page_range[0], page_range[1]):
        page_text = pdfread.getPage(i).extractText().replace('\n', ' ')
        matches = re.findall(look_ahead, page_text)
        last = re.sub(look_ahead, '', page_text)
        last = re.search(pattern + r'(.+)', last)
        matches = matches + [(last.group(dep_group), last.group(abbrev_group), 
                    last.group(c_group), last.group(t_group), last.group(5))]
        
        for match in matches:  
            # Getting full department name
            if match[dep_group - 1] != '':
                dep_names[match[abbrev_group - 1]] = match[dep_group - 1]
            dep_name = dep_names.get(match[abbrev_group - 1])
            
            # Other groups
            course_num = int(match[c_group - 1][:3])
            title = str(match[t_group - 1])
            desc = str(match[4]) # assuming description is group 5
            
            #Filtering
            for key in FS_keys:
                if (re.search(key, title, flags=re.IGNORECASE) or 
                    re.search(key, desc, flags=re.IGNORECASE)):
                    G_UG = grad_status
                    if grad_status:
                        if type(grad_status) == int:
                            G_UG = "UG"
                            if int(t.group(1)) >= grad_status:
                                G_UG = "G"
                    lab = False
                    if (re.search(r'Laboratory|Lab', title, flags=re.IGNORECASE) or 
                    re.search(r'Laboratory|Lab', desc, flags=re.IGNORECASE)):
                        lab = True
                    csv_writer.writerow([dep_name, course_num, title, desc, 
                                             G_UG, None, lab, year])
                                            # Leaving format blank for now
                
dine_pattern = r'([a-zA-Z]+\s\(\w{2,3}\))?\s+([\/a-zA-Z]+)\s([\d]+\w?)\s([^\d]+)\(\d\)\s+'
#Partial Test set:
get_data('Dine_College_Sample.csv', "dinecollege2019-2020.pdf", [19, 94], dine_pattern, 
         1, 2, 3, 4, '2019-2020', None)

In [48]:
pd.read_csv('Dine_College_Sample.csv')

,Department Name,Course Catalogue Number,Course Name,Course Description,Graduate/Undergraduate,Format,Lab,Academic Catalogue Year
0,AGRICULTURE (AGR),105,Int roduction to Animal Science,This course will introduce students to animal ...,NaN,NaN,False,2019-2020
1,NaN,160,Navajo Agricult ural History to Present,Note: Cross - listed with Nis 160. ANT 160...,NaN,NaN,False,2019-2020
2,NaN,385,Comparative Invertebrate Zoology,Prerequisite: BIO 181. This course involves ...,NaN,NaN,True,2019-2020
3,NaN,425,Animal Physiology,Prerequisite or Co - requisite: BIO 344. Th...,NaN,NaN,True,2019-2020
4,ECONOMICS (ECO),460,Tribal Economic Issues and Development Strateg...,"Prerequisite: MGT 420, BUS 460. This is a ba...",NaN,NaN,True,2019-2020
5,TECHNOLOGY (ENV),101,Introduction to Environmental Science,"Prerequisite: ENG 075, 085 , MTH 096. This c...",NaN,NaN,True,2019-2020
6,TECHNOLOGY (ENV),110,Introduction to Soils/Lab,Prerequisite: ENV 101. This course introduce...,NaN,NaN,True,2019-2020
7,NaN,225,Principles of Agroecology,Prerequisite: ENV 101 or BIO 182. This cours...,NaN,NaN,False,2019-2020
8,NaN,450,Diné Resource Management,Prerequisite: NIS 420. This course will ex...,NaN,NaN,False,2019-2020
9,HEALTH (PUH),141,Nutrition for Health,Prerequisite: ENG 085 and MTH 096. General c...,NaN,NaN,False,2019-2020


In [23]:
sample = '20     COURSE  PREFIX  CODE,  TITLE AND COURSE DES CRIPTION   ACCOUNTING (ACC)   ACC 100 Fundamentals of Accounting (3)   Prerequisite:     This course is an introduction to accounting fundamentals. It includes journalizing, posting, worksheets,  financial reports, and data processing for accounting. One hour laboratory required.   ACC 200 Financial Accounting (3)      This course covers the theory and practice of accounting in services and merchandising firms. It  emphasizes the accounting cycle, periodic reporting, receivables, payables, inventories, and accounting  systems . One hour laboratory required.   ACC 201 Managerial Accounting (3)      This course is a continuation of ACC 200 with emphasis on manufacturing firms, and managerial  approaches for decision - making. One hour laborato ry required.   ACC 310 Government and Non - Profit   Accounting (3)   Prerequisite: Minimu  .   This course covers the theory and practice of accounting for governmental and not - for - profit units.  Topics include financial reporting; budgetary a ccounting; fund accounts; and nonprofit accounting for  entities such as public colleges, universities and health care providers .   ACC 350 Finance (3)    ACC 310,  BUS 265, ECO 200 or 201.   Introduction to the theories and applications associated with the functions of capital markets. This  includes the conceptual foundations of portfolio theory, risk management, and asset valuation. This  course will examine the markets for equity, money, and  bonds. One hour laboratory required.   AGRICULTURE (AGR)   AGR 103 Conservation Planning (4)   This course introduces students to the understanding natural resource conservation issues and concerns  to allow them to develop conservation plans necessary to improve   natural resource conditions applicable  to tribal lands.   AGR 105 Int roduction to Animal Science (4)   This course will introduce students to animal and livestock industries, production, systems and markets.  Students will not only learn about national product ion but include Navajo Nation livestock industry.   AGR 110 Monitoring/Assessments and Rangeland Management (4)   This course will introduce students to the various grazing management concept, identify local vegetation  communities, identification of local plan ts, as related to current Navajo producers grazing management  systems. Field trips and field works will serve as a lab for hands - on and field experience to learn range  and vegetation monitoring concepts.   '
rest = re.sub(dine_pattern + r'(.+?)(?=' + dine_pattern + r')', '',sample)
rest

'20     COURSE  PREFIX  CODE,  TITLE AND COURSE DES CRIPTION      AGR 110 Monitoring/Assessments and Rangeland Management (4)   This course will introduce students to the various grazing management concept, identify local vegetation  communities, identification of local plan ts, as related to current Navajo producers grazing management  systems. Field trips and field works will serve as a lab for hands - on and field experience to learn range  and vegetation monitoring concepts.   '

In [34]:
rest_match = re.search(dine_pattern + r'(.+)', rest)
print(rest_match.group(1))
print(type(rest_match.group(2)))
print(rest_match.group(3))
print(rest_match.group(4))
print(rest_match.group(5))

None
<class 'str'>
110
Monitoring/Assessments and Rangeland Management 
This course will introduce students to the various grazing management concept, identify local vegetation  communities, identification of local plan ts, as related to current Navajo producers grazing management  systems. Field trips and field works will serve as a lab for hands - on and field experience to learn range  and vegetation monitoring concepts.   


In [15]:
sample = '20     COURSE  PREFIX  CODE,  TITLE AND COURSE DES CRIPTION   ACCOUNTING (ACC)   ACC 100 Fundamentals of Accounting (3)   Prerequisite:     This course is an introduction to accounting fundamentals. It includes journalizing, posting, worksheets,  financial reports, and data processing for accounting. One hour laboratory required.   ACC 200 Financial Accounting (3)      This course covers the theory and practice of accounting in services and merchandising firms. It  emphasizes the accounting cycle, periodic reporting, receivables, payables, inventories, and accounting  systems . One hour laboratory required.   ACC 201 Managerial Accounting (3)      This course is a continuation of ACC 200 with emphasis on manufacturing firms, and managerial  approaches for decision - making. One hour laborato ry required.   ACC 310 Government and Non - Profit   Accounting (3)   Prerequisite: Minimu  .   This course covers the theory and practice of accounting for governmental and not - for - profit units.  Topics include financial reporting; budgetary a ccounting; fund accounts; and nonprofit accounting for  entities such as public colleges, universities and health care providers .   ACC 350 Finance (3)    ACC 310,  BUS 265, ECO 200 or 201.   Introduction to the theories and applications associated with the functions of capital markets. This  includes the conceptual foundations of portfolio theory, risk management, and asset valuation. This  course will examine the markets for equity, money, and  bonds. One hour laboratory required.   AGRICULTURE (AGR)   AGR 103 Conservation Planning (4)   This course introduces students to the understanding natural resource conservation issues and concerns  to allow them to develop conservation plans necessary to improve   natural resource conditions applicable  to tribal lands.   AGR 105 Int roduction to Animal Science (4)   This course will introduce students to animal and livestock industries, production, systems and markets.  Students will not only learn about national product ion but include Navajo Nation livestock industry.   AGR 110 Monitoring/Assessments and Rangeland Management (4)   This course will introduce students to the various grazing management concept, identify local vegetation  communities, identification of local plan ts, as related to current Navajo producers grazing management  systems. Field trips and field works will serve as a lab for hands - on and field experience to learn range  and vegetation monitoring concepts.   '
dine_pattern = r'(\w+\s\(\w{2,3}\))?\s+([a-zA-Z]{2,3})\s([\d]+\w?)\s([^\d]+)\(\d\)\s+(.+?)'
last = re.sub(dine_pattern + '(?=' + dine_pattern + ')', '', sample)
l

'20     COURSE  PREFIX  CODE,  TITLE AND COURSE DES CRIPTION      AGR 110 Monitoring/Assessments and Rangeland Management (4)   This course will introduce students to the various grazing management concept, identify local vegetation  communities, identification of local plan ts, as related to current Navajo producers grazing management  systems. Field trips and field works will serve as a lab for hands - on and field experience to learn range  and vegetation monitoring concepts.   '

In [9]:
r'test' + r'test2'

'testtest2'

In [15]:
dine_pattern + r'(?=' + dine_pattern + r')'

'(\\w+\\s\\(\\w{2,3}\\))?\\s+([a-zA-Z]{2,3})\\s([\\d]+\\w?)\\s([^\\d]+)\\(\\d\\)\\s+(.+?)(?=(\\w+\\s\\(\\w{2,3}\\))?\\s+([a-zA-Z]{2,3})\\s([\\d]+\\w?)\\s([^\\d]+)\\(\\d\\)\\s+(.+?))'

In [33]:
sample
re.findall(dine_pattern + r'(.+)(?=' + dine_pattern + r')', sample)[0]

('ACCOUNTING (ACC)',
 'ACC',
 '100',
 'Fundamentals of Accounting ',
 'Prerequisite:     This course is an introduction to accounting fundamentals. It includes journalizing, posting, worksheets,  financial reports, and data processing for accounting. One hour laboratory required.   ACC 200 Financial Accounting (3)      This course covers the theory and practice of accounting in services and merchandising firms. It  emphasizes the accounting cycle, periodic reporting, receivables, payables, inventories, and accounting  systems . One hour laboratory required.   ACC 201 Managerial Accounting (3)      This course is a continuation of ACC 200 with emphasis on manufacturing firms, and managerial  approaches for decision - making. One hour laborato ry required.   ACC 310 Government and Non - Profit   Accounting (3)   Prerequisite: Minimu  .   This course covers the theory and practice of accounting for governmental and not - for - profit units.  Topics include financial reporting; budgetary a